<a href="https://colab.research.google.com/github/hriteshMaikap/personalised-github-repo-recommendation/blob/main/Recommendation_Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers scikit-learn torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

Preprocessing Readme

In [ ]:
# Importing necessary libraries
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Load the dataset (assuming it is in CSV format and uploaded to Colab)
df = pd.read_csv('/content/final.csv')

# Display the first few rows of the dataframe
print(df.head())

# Preprocessing function
def preprocess(text):
    # Basic preprocessing: lowercasing, removing special characters, etc.
    text = text.lower().replace('\n', ' ').replace('\r', '')
    return text

# Apply preprocessing to the readme column
df['readme'] = df['readme'].apply(preprocess)

# Display the preprocessed dataframe
print(df.head())


                  name                           full_name  stars  \
0     Firefox-Mod-Blur         datguypiko/Firefox-Mod-Blur   1053   
1     obsidian-minimal             kepano/obsidian-minimal   3398   
2                 kado                   septiandwica/kado    246   
3  firefox-gnome-theme  rafaelmardojai/firefox-gnome-theme   3248   
4           AnuPpuccin             AnubisNekhet/AnuPpuccin   1845   

                        languages  \
0                         ['CSS']   
1   ['CSS', 'SCSS', 'JavaScript']   
2         ['CSS', 'SCSS', 'HTML']   
3  ['CSS', 'Shell', 'JavaScript']   
4     ['CSS', 'SCSS', 'Makefile']   

                                              topics  \
0  ['css', 'userchrome', 'firefox', 'theme', 'fir...   
1  ['obsidian', 'obsidian-md', 'obsidian-theme', ...   
2                                                 []   
3  ['gnome', 'theme', 'firefox', 'userchrome', 'c...   
4               ['obsidian', 'obsidian-md', 'theme']   

                         

TF-IDF to extract keywords

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Function to extract top keywords using TF-IDF
def extract_keywords_tfidf(corpus, top_n=20):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
    X = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names_out()

    # Function to extract top keywords for a single document
    def get_top_keywords(tfidf_vector, feature_names, top_n):
        sorted_indices = tfidf_vector.argsort()[::-1][:top_n]
        top_keywords = [feature_names[i] for i in sorted_indices]
        return top_keywords

    top_keywords_list = []
    for row in X.toarray():
        top_keywords_list.append(get_top_keywords(row, feature_names, top_n))

    return top_keywords_list

# Combine README and issues for each repository
corpus = df['readme'].tolist()

# Extract keywords using TF-IDF for each document in the corpus
df['keywords_tfidf'] = extract_keywords_tfidf(corpus)

# Display the dataframe with the new keywords column
print(df[['readme', 'keywords_tfidf']].head())


                                              readme  \
0  firefox mod blur - modular :wrench: version 3....   
1  find detailed documentation at [minimal.guide]...   
2  # kado -flower code from: # description flower...   
3  # firefox gnome theme []( []( []( **a gnome th...   
4  anuppuccin personal theme for obsidian :3 vote...   

                                      keywords_tfidf  
0  [css, firefox, gear, tabs, extra, mod, 2023, t...  
1  [cards, minimal, width, plugin, table, setting...  
2  [flower, mdusmanansari, css, code, tiktok, di,...  
3  [firefox, theme, gnometheme, tab, gnome, icons...  
4  [theme, yes, anuppuccin, bubble, speech, snipp...  


Printing the keywords

In [ ]:
# Convert the 'keywords_tfidf' column to a list
keywords_tfidf_list = df['keywords_tfidf'].tolist()

# Print all entries in the 'keywords_tfidf' column
for idx, keywords in enumerate(keywords_tfidf_list):
    print(f"Entry {idx + 1}: {keywords}")


Entry 1: ['css', 'firefox', 'gear', 'tabs', 'extra', 'mod', '2023', 'theme', 'v2', 'micaforeveryone', 'fix', 'buttons', 'menu', 'v117', 'tested', 'modification', 'bar', 'v115', 'added', 'tab']
Entry 2: ['cards', 'minimal', 'width', 'plugin', 'table', 'settings', 'tables', 'schemes', 'obsidian', 'hotkeys', 'color', 'helper', 'theme', 'classes', 'plugins', 'images', 'image', 'borders', 'dataview', 'mode']
Entry 3: ['flower', 'mdusmanansari', 'css', 'code', 'tiktok', 'di', 'responsive', 'author', 'index', 'style', 'description', 'thanks', 'web', 'md', 'file', 'frequency', 'fresh', 'fu', 'frequently', 'frequent']
Entry 4: ['firefox', 'theme', 'gnometheme', 'tab', 'gnome', 'icons', 'css', 'sh', 'tabs', 'mozilla', 'extensions', 'preferences', 'script', 'button', 'scripts', 'install', 'profile', 'checkout', 'adwaita', 'headerbar']
Entry 5: ['theme', 'yes', 'anuppuccin', 'bubble', 'speech', 'snippet', 'colorschemes', 'rainbow', 'style', 'palette', 'mocha', 'pine', 'custom', 'nord', 'ros', 'dar

Applying LDA to extract keywords

In [ ]:
# LDA for topic extraction
def extract_keywords_lda(corpus, n_topics=2):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
    X = vectorizer.fit_transform(corpus)
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
    lda.fit(X)
    feature_names = vectorizer.get_feature_names_out()
    keywords = []
    for topic_idx, topic in enumerate(lda.components_):
        top_keywords = [feature_names[i] for i in topic.argsort()[:-21:-1]]
        keywords.extend(top_keywords)
    return keywords

# Extract keywords using LDA
keywords_lda = extract_keywords_lda(corpus)
df['keywords_lda'] = df['readme'].apply(lambda x: extract_keywords_lda([x]))


Listing keywords for LDA

In [ ]:
# Convert the 'keywords_tfidf' column to a list
keywords_lda_list = df['keywords_lda'].tolist()

# Print all entries in the 'keywords_tfidf' column
for idx, keywords in enumerate(keywords_lda_list):
    print(f"Entry {idx + 1}: {keywords}")


Entry 1: ['css', 'firefox', '2023', 'extra', 'v2', 'theme', 'fix', 'new', 'file', 'mod', 'tabs', 'added', 'tested', 'menu', 'buttons', 'search', 'gear', 'bar', 'style', 'folder', 'css', 'firefox', '2023', 'extra', 'theme', 'v2', 'new', 'file', 'fix', 'mod', 'tabs', 'buttons', 'added', 'tested', 'menu', 'gear', 'search', 'style', 'bar', 'tab']
Entry 2: ['minimal', 'table', 'plugin', 'cards', 'settings', 'width', 'images', 'image', 'color', 'plugins', 'theme', 'add', 'tables', 'mode', 'classes', 'line', 'use', 'schemes', 'support', 'obsidian', 'minimal', 'table', 'settings', 'cards', 'plugin', 'width', 'images', 'color', 'image', 'plugins', 'add', 'mode', 'theme', 'tables', 'classes', 'line', 'use', 'schemes', 'support', 'using']
Entry 3: ['code', 'flower', 'mdusmanansari', 'css', 'septiandwica', 'kado', 'di', 'disesuikan', 'ansari', 'md', 'bisa', 'author', 'codepan', 'description', 'dwi', 'index', 'style', 'usman', 'thanks', 'file', 'code', 'flower', 'css', 'mdusmanansari', 'septian', '

Keyword extraction using BERT

In [ ]:
!pip install keybert
!pip install transformers

import pandas as pd
from keybert import KeyBERT
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')
def extract_keywords_bert(text):
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=10)
    return [kw[0] for kw in keywords]

# Extract keywords using KeyBERT for each repository
df['keywords_bert'] = df['readme'].apply(extract_keywords_bert)

# Display the dataframe with the new keywords column
print(df.head())


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.0 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.4-py3-none-any.whl size=39200 sha256=5081d72d705f6df29a8fdb63d6cec3c9730d9cd8735792b027c7115fb977ed9b
  Stored in directory: /root/.cache/pip/wheels/97/ef/4c/6588bd7072b0cc04225b40e639b991e49ebd4e21fb81f0acee
Successfully built keybert


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

                  name                           full_name  stars  \
0     Firefox-Mod-Blur         datguypiko/Firefox-Mod-Blur   1053   
1     obsidian-minimal             kepano/obsidian-minimal   3398   
2                 kado                   septiandwica/kado    246   
3  firefox-gnome-theme  rafaelmardojai/firefox-gnome-theme   3248   
4           AnuPpuccin             AnubisNekhet/AnuPpuccin   1845   

                        languages  \
0                         ['CSS']   
1   ['CSS', 'SCSS', 'JavaScript']   
2         ['CSS', 'SCSS', 'HTML']   
3  ['CSS', 'Shell', 'JavaScript']   
4     ['CSS', 'SCSS', 'Makefile']   

                                              topics  \
0  ['css', 'userchrome', 'firefox', 'theme', 'fir...   
1  ['obsidian', 'obsidian-md', 'obsidian-theme', ...   
2                                                 []   
3  ['gnome', 'theme', 'firefox', 'userchrome', 'c...   
4               ['obsidian', 'obsidian-md', 'theme']   

                         

Printing BERT Extracted Keywords

In [ ]:
# Convert the 'keywords_tfidf' column to a list
keywords_bert_list = df['keywords_bert'].tolist()

# Print all entries in the 'keywords_tfidf' column
for idx, keywords in enumerate(keywords_lda_list):
    print(f"Entry {idx + 1}: {keywords}")

Entry 1: ['css', 'firefox', '2023', 'extra', 'v2', 'theme', 'fix', 'new', 'file', 'mod', 'tabs', 'added', 'tested', 'menu', 'buttons', 'search', 'gear', 'bar', 'style', 'folder', 'css', 'firefox', '2023', 'extra', 'theme', 'v2', 'new', 'file', 'fix', 'mod', 'tabs', 'buttons', 'added', 'tested', 'menu', 'gear', 'search', 'style', 'bar', 'tab']
Entry 2: ['minimal', 'table', 'plugin', 'cards', 'settings', 'width', 'images', 'image', 'color', 'plugins', 'theme', 'add', 'tables', 'mode', 'classes', 'line', 'use', 'schemes', 'support', 'obsidian', 'minimal', 'table', 'settings', 'cards', 'plugin', 'width', 'images', 'color', 'image', 'plugins', 'add', 'mode', 'theme', 'tables', 'classes', 'line', 'use', 'schemes', 'support', 'using']
Entry 3: ['code', 'flower', 'mdusmanansari', 'css', 'septiandwica', 'kado', 'di', 'disesuikan', 'ansari', 'md', 'bisa', 'author', 'codepan', 'description', 'dwi', 'index', 'style', 'usman', 'thanks', 'file', 'code', 'flower', 'css', 'mdusmanansari', 'septian', '

Creating list of final keywords

In [ ]:
# Add the final keyword set as a column in the DataFrame
df['final_keywords'] = df.apply(lambda row: list(set(row['keywords_bert'] + row['keywords_tfidf'] + row['keywords_lda'])), axis=1)

keywords_list = df['final_keywords'].tolist()

# Print all entries in the 'keywords_tfidf' column
for idx, keywords in enumerate(keywords_lda_list):
    print(f"Entry {idx + 1}: {keywords}")


Entry 1: ['css', 'firefox', '2023', 'extra', 'v2', 'theme', 'fix', 'new', 'file', 'mod', 'tabs', 'added', 'tested', 'menu', 'buttons', 'search', 'gear', 'bar', 'style', 'folder', 'css', 'firefox', '2023', 'extra', 'theme', 'v2', 'new', 'file', 'fix', 'mod', 'tabs', 'buttons', 'added', 'tested', 'menu', 'gear', 'search', 'style', 'bar', 'tab']
Entry 2: ['minimal', 'table', 'plugin', 'cards', 'settings', 'width', 'images', 'image', 'color', 'plugins', 'theme', 'add', 'tables', 'mode', 'classes', 'line', 'use', 'schemes', 'support', 'obsidian', 'minimal', 'table', 'settings', 'cards', 'plugin', 'width', 'images', 'color', 'image', 'plugins', 'add', 'mode', 'theme', 'tables', 'classes', 'line', 'use', 'schemes', 'support', 'using']
Entry 3: ['code', 'flower', 'mdusmanansari', 'css', 'septiandwica', 'kado', 'di', 'disesuikan', 'ansari', 'md', 'bisa', 'author', 'codepan', 'description', 'dwi', 'index', 'style', 'usman', 'thanks', 'file', 'code', 'flower', 'css', 'mdusmanansari', 'septian', '

Saving it to the file

In [ ]:
# Define the file path for the new CSV file
output_csv_file = 'repoWithKeywords.csv'

# Save the DataFrame to a new CSV file
df.to_csv(output_csv_file, index=False)

print(f'Data saved to {output_csv_file}')


Data saved to repoWithKeywords.csv


Vectorizing Data

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the data
df = pd.read_csv('/content/repoWithKeywords.csv')

# Normalize the 'stars' column
scaler = MinMaxScaler()
df[['stars']] = scaler.fit_transform(df[['stars']])

# Encode 'languages' and 'topics' using MultiLabelBinarizer
mlb_languages = MultiLabelBinarizer()
mlb_topics = MultiLabelBinarizer()

df['languages'] = df['languages'].apply(lambda x: eval(x))  # Convert string representation of list to actual list
df['topics'] = df['topics'].apply(lambda x: eval(x))

languages_encoded = mlb_languages.fit_transform(df['languages'])
topics_encoded = mlb_topics.fit_transform(df['topics'])

languages_df = pd.DataFrame(languages_encoded, columns=mlb_languages.classes_)
topics_df = pd.DataFrame(topics_encoded, columns=mlb_topics.classes_)

# Vectorize 'issues' and 'final_keywords' using TF-IDF
tfidf_vectorizer_issues = TfidfVectorizer()
tfidf_vectorizer_keywords = TfidfVectorizer()

df['issues'] = df['issues'].apply(lambda x: x.replace(", ", " "))
df['final_keywords'] = df['final_keywords'].apply(lambda x: " ".join(eval(x)))

issues_tfidf = tfidf_vectorizer_issues.fit_transform(df['issues'])
final_keywords_tfidf = tfidf_vectorizer_keywords.fit_transform(df['final_keywords'])

issues_df = pd.DataFrame(issues_tfidf.toarray(), columns=tfidf_vectorizer_issues.get_feature_names_out())
final_keywords_df = pd.DataFrame(final_keywords_tfidf.toarray(), columns=tfidf_vectorizer_keywords.get_feature_names_out())

# Combine all features into a single DataFrame
preprocessed_df = pd.concat([df[['name', 'full_name', 'stars']], languages_df, topics_df, issues_df, final_keywords_df], axis=1)

# Save preprocessed data to CSV
preprocessed_df.to_csv('repo_vector.csv', index=False)


Similarity Index Matching

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load the data
df1 = pd.read_csv('/content/repo_vector.csv')
df2 = pd.read_csv('/content/user_tfidf_vectors.csv')

def compute_cosine_similarity(user_vector, repo_vectors):
    # Identify common features
    common_features = user_vector.index.intersection(repo_vectors.columns)

    # Filter vectors based on common features
    user_vector_filtered = user_vector[common_features].values.reshape(1, -1)
    repo_vectors_filtered = repo_vectors[common_features].values

    # Compute cosine similarity
    similarity_scores = cosine_similarity(user_vector_filtered, repo_vectors_filtered)

    return similarity_scores.flatten()

# Assuming the 'name' and 'full_name' columns are the first two columns in df1,
# and 'username' is the first column in df2.
repo_identifiers = df1[['name', 'full_name']]
user_identifiers = df2[['username']]

# Extract feature vectors
repo_features = df1.iloc[:, 2:]  # Exclude 'name' and 'full_name'
user_features = df2.iloc[:, 1:]  # Exclude 'username'

# Suggest top 5 repositories for each user
top_n = 5
recommendations = []

for idx, user in user_identifiers.iterrows():
    user_vector = user_features.iloc[idx]
    similarity_scores = compute_cosine_similarity(user_vector, repo_features)

    top_repos_indices = similarity_scores.argsort()[-top_n:][::-1]
    top_repos = repo_identifiers.iloc[top_repos_indices]
    top_scores = similarity_scores[top_repos_indices]

    recommendations.append({
        'username': user['username'],
        'recommendations': list(zip(top_repos['name'], top_repos['full_name'], top_scores))
    })

# Print recommendations
for recommendation in recommendations:
    print(f"User: {recommendation['username']}")
    for repo_name, repo_full_name, score in recommendation['recommendations']:
        print(f"  Repository: {repo_name}, Full Name: {repo_full_name}, Similarity Score: {score:.4f}")


User: brson
  Repository: flink-cdc, Full Name: apache/flink-cdc, Similarity Score: 0.4578
  Repository: purchase-workflow, Full Name: OCA/purchase-workflow, Similarity Score: 0.3621
  Repository: web, Full Name: OCA/web, Similarity Score: 0.3262
  Repository: samples, Full Name: flutter/samples, Similarity Score: 0.3208
  Repository: pyroscope, Full Name: grafana/pyroscope, Similarity Score: 0.3088
User: ItzCrazyKns
  Repository: pandas, Full Name: pandas-dev/pandas, Similarity Score: 0.5516
  Repository: OpenArk, Full Name: BlackINT3/OpenArk, Similarity Score: 0.5057
  Repository: blurhash, Full Name: woltapp/blurhash, Similarity Score: 0.4813
  Repository: Telegram, Full Name: DrKLO/Telegram, Similarity Score: 0.4346
  Repository: Spoon-Knife, Full Name: octocat/Spoon-Knife, Similarity Score: 0.3342
User: ua741
  Repository: design-blocks, Full Name: froala/design-blocks, Similarity Score: 0.4268
  Repository: KernelSU, Full Name: tiann/KernelSU, Similarity Score: 0.4096
  Repositor

Clustering Similarity Matching

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the data
df1 = pd.read_csv('/content/repo_vector.csv')
df2 = pd.read_csv('/content/user_tfidf_vectors.csv')

def cluster_repositories(repo_vectors, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    kmeans.fit(repo_vectors)
    clusters = kmeans.labels_
    return kmeans, clusters

def find_closest_cluster(user_vector, cluster_centers):
    similarities = cosine_similarity(user_vector.reshape(1, -1), cluster_centers)
    closest_cluster = np.argmax(similarities)
    return closest_cluster

# Assuming the 'name' and 'full_name' columns are the first two columns in df1,
# and 'username' is the first column in df2.
repo_identifiers = df1[['name', 'full_name']]
user_identifiers = df2[['username']]

# Extract feature vectors
repo_features = df1.iloc[:, 2:]  # Exclude 'name' and 'full_name'
user_features = df2.iloc[:, 1:]  # Exclude 'username'

# Identify common features
common_features = repo_features.columns.intersection(user_features.columns)

# Filter vectors based on common features
repo_features_filtered = repo_features[common_features]
user_features_filtered = user_features[common_features]

# Cluster the repositories
num_clusters = len(user_identifiers)  # One cluster per user
kmeans, clusters = cluster_repositories(repo_features_filtered.values, num_clusters)

# Add cluster labels to the repo_identifiers dataframe
repo_identifiers.loc[:, 'cluster'] = clusters

# Suggest top 5 repositories for each user
recommendations = []

for idx, user in user_identifiers.iterrows():
    user_vector = user_features_filtered.iloc[idx].values
    closest_cluster = find_closest_cluster(user_vector, kmeans.cluster_centers_)

    # Get repositories in the closest cluster
    cluster_repos = repo_identifiers[repo_identifiers['cluster'] == closest_cluster]

    # Compute similarity scores within the cluster
    cluster_repo_features = repo_features_filtered.iloc[cluster_repos.index]
    similarity_scores = cosine_similarity(user_vector.reshape(1, -1), cluster_repo_features.values).flatten()

    top_n = 5
    top_repos_indices = similarity_scores.argsort()[-top_n:][::-1]
    top_repos = cluster_repos.iloc[top_repos_indices]
    top_scores = similarity_scores[top_repos_indices]

    recommendations.append({
        'username': user['username'],
        'recommendations': list(zip(top_repos['name'], top_repos['full_name'], top_scores))
    })

# Print recommendations
for recommendation in recommendations:
    print(f"User: {recommendation['username']}")
    for repo_name, repo_full_name, score in recommendation['recommendations']:
        print(f"  Repository: {repo_name}, Full Name: {repo_full_name}, Similarity Score: {score:.4f}")


User: brson
  Repository: flink-cdc, Full Name: apache/flink-cdc, Similarity Score: 0.4578
  Repository: purchase-workflow, Full Name: OCA/purchase-workflow, Similarity Score: 0.3621
  Repository: web, Full Name: OCA/web, Similarity Score: 0.3262
  Repository: samples, Full Name: flutter/samples, Similarity Score: 0.3208
  Repository: pyroscope, Full Name: grafana/pyroscope, Similarity Score: 0.3088
User: ItzCrazyKns
  Repository: pandas, Full Name: pandas-dev/pandas, Similarity Score: 0.5516
  Repository: blurhash, Full Name: woltapp/blurhash, Similarity Score: 0.4813
  Repository: Telegram, Full Name: DrKLO/Telegram, Similarity Score: 0.4346
  Repository: Spoon-Knife, Full Name: octocat/Spoon-Knife, Similarity Score: 0.3342
  Repository: BCR, Full Name: chenxiaolong/BCR, Similarity Score: 0.2609
User: ua741
  Repository: atuin, Full Name: atuinsh/atuin, Similarity Score: 0.3938
  Repository: khoj, Full Name: khoj-ai/khoj, Similarity Score: 0.3707
  Repository: melos, Full Name: inver

<ipython-input-72-eb4c4a08d6c3>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repo_identifiers.loc[:, 'cluster'] = clusters
